<a href="https://colab.research.google.com/github/roquearmas/IABD_DATA/blob/main/TresnRaya_Qlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hay que entrenar un segundo jugador, para poder evaluar que, al final, deben empatar siempre.

In [140]:
def estado_valido (numero, jugador):

# hay que sacar solamente los numeros 0, 1 o 2
# solamente valen los que tengan igual número de dos que de unos
# o que tengan un 1 más que un dos, pero no más

  unos=contar_repeticiones(numero,1)
  doses=contar_repeticiones(numero,2)

  if jugador ==1:
    if doses==unos :
      return True
    else:
      return False
  else:
    if doses + 1==unos:
      return True
    else:
      return False



In [ ]:
def contar_repeticiones(numero, digito):
  contador = 0
  for caracter in str(numero):
    if caracter == str(digito):
      contador += 1
  return contador

In [141]:
Q=[]
R=[]
dicc=dict()
dicc_inver=dict()
indice=0
for a in range(3):
  for b in range(3):
    for c in range(3):
      for d in range(3):
        for e in range(3):
          for f in range(3):
            for g in range(3):
              for h in range(3):
                for i in range(3):
                  ganador_1 = False
                  ganador_2 = False
                  estado =int(str(a)+str(b)+str(c)+str(d)+str(e)+str(f)+str(g)+str(h)+str(i))

                  if estado_valido(estado,1) or estado_valido(estado,2):

                    ganador_1 = ((a*b*c==1) or (d*e*f==1) or (g*h*i==1) or (a*e*i==1) or (c*e*g==1) or (a*d*g==1) or (b*e*h==1) or (c*f*i==1))
                    ganador_2 = ((a*b*c==8) or (d*e*f==8) or (g*h*i==8) or (a*e*i==8) or (c*e*g==8) or (a*d*g==8) or (b*e*h==8) or (c*f*i==8))

                    if not(ganador_1 & ganador_2):
                      indice_cod = a*(3**8) + b*(3**7) + c*(3**6) + d*(3**5) + e*(3**4) + f*(3**3) + g*(3**2) + h*3 + i
                      dicc[indice_cod] = indice
                      dicc_inver[indice] = indice_cod
                      Q.append(indice_cod)
                      indice +=1
                    # detectar las jugadas ganadoras para 1 y darles recompensa 100
                    if ganador_1:
                      R.append (100)
                    elif ganador_2:
                      R.append (-100)
                    # detectar los empates y darles recompensa de 10
                    elif a*b*c*d*e*f*g*h*i !=0: # empate
                      R.append (10)
                    else:
                      R.append (0)



Método para pasar estado decimal a base 3


In [142]:
def decodificar_estado(numero):
  return np.base_repr(numero, base=3).zfill(9)



Método que detecta si en un estado hay dos ganadores

In [143]:
def dos_ganadores(estado):
  a=int(estado[0])
  b=int(estado[1])
  c=int(estado[2])
  d=int(estado[3])
  e=int(estado[4])
  f=int(estado[5])
  g=int(estado[6])
  h=int(estado[7])
  i=int(estado[8])
  ganador_1 = ((a*b*c==1) or (d*e*f==1) or (g*h*i==1) or (a*e*i==1) or (c*e*g==1) or (a*d*g==1) or (b*e*h==1) or (c*f*i==1))
  ganador_2 = ((a*b*c==8) or (d*e*f==8) or (g*h*i==8) or (a*e*i==8) or (c*e*g==8) or (a*d*g==8) or (b*e*h==8) or (c*f*i==8))
  return ganador_1 & ganador_2


método que detecta un estado_ganador

In [144]:
def estado_ganador(estado):
  a=int(estado[0])
  b=int(estado[1])
  c=int(estado[2])
  d=int(estado[3])
  e=int(estado[4])
  f=int(estado[5])
  g=int(estado[6])
  h=int(estado[7])
  i=int(estado[8])
  ganador_1 = ((a*b*c==1) or (d*e*f==1) or (g*h*i==1) or (a*e*i==1) or (c*e*g==1) or (a*d*g==1) or (b*e*h==1) or (c*f*i==1))
  ganador_2 = ((a*b*c==8) or (d*e*f==8) or (g*h*i==8) or (a*e*i==8) or (c*e*g==8) or (a*d*g==8) or (b*e*h==8) or (c*f*i==8))
  return ganador_1 or ganador_2


In [145]:
def estado_imposible(matrizT,indice):
  for elemento in matrizT[indice]:
    if elemento != -1:
      return False
  print('imposible', indice,matrizT[indice] )
  return True


Creo la matriz de transiciones, para el jugador 1

In [146]:
import numpy as np

T= np.zeros ((len(Q),9),dtype=int)
for i in range (len(Q)): #estados posibles
#for i in range(10):
    # decodificar el estado
    estadoQ=Q[i]
    estado_decod=decodificar_estado(estadoQ)
    ceros= contar_repeticiones(estado_decod,0)
    if ceros==0:
      # no existen ceros en el estado, no se puede jugar. Todas las acciones son -1
      for k in range(9)  :
        T[i,k]=-1

    else: #existen ceros, hay jugada
      for j in range (9):
        str_estado=str(estado_decod)

        if str_estado [j]=='0':
          str_estado = str_estado[:j]+'1'+ str_estado[j+1:] # tenemos la jugada hecha para el jugador uno
          if estado_valido(int(str_estado),1):
            if estado_ganador(str_estado)==False: # si detectamos que uno de los dos jugadores ganó, es un estado final. No va a ningún otro estado
              indice_cod=0
              for k in range(9):
                indice_cod +=int( str_estado[k])*(3**(8-k))

              #buscamos el estado en la tabla Q, con el índice del estado resultante.
              T[i,j]=dicc[indice_cod]
            else:
              T[i,j]=-1
          else:
            T[i,j]=-1
        else :
          T[i,j]=-1
        # print (i,estado_decod, str_estado, T[i,j])



Y la del jugador 2

In [147]:
T2= np.zeros ((len(Q),9),dtype=int)
for i in range (len(Q)): #estados posibles
#for i in range(10):
    # decodificar el estado
    estadoQ=Q[i]
    estado_decod=decodificar_estado(estadoQ)
    ceros= contar_repeticiones(estado_decod,0)
    if ceros==0:
      # no existen ceros en el estado, no se puede jugar. Todas las acciones son -1
      for k in range(9)  :
        T2[i,k]=-1

    else: #existen ceros, hay jugada
      for j in range (9):
        str_estado=str(estado_decod)

        if str_estado [j]=='0':
          str_estado = str_estado[:j]+'2'+ str_estado[j+1:] # tenemos la jugada hecha para el jugador dos
          if estado_valido(int(str_estado),2):
            if estado_ganador(str_estado)==False: # si detectamos que uno de los dos jugadores ganó, es un estado final. No va a ningún otro estado
              indice_cod=0
              for k in range(9):
                indice_cod +=int( str_estado[k])*(3**(8-k))
                # print (str_estado,int( str_estado[k]),k,3**(8-k),indice_cod,int( str_estado[k])*(3**(8-k)) )
                # print (indice_cod)
              #buscamos el estado en la tabla Q, con el índice del estado resultante.
              #print (i,j,indice_cod)
              T2[i,j]=dicc[indice_cod]
            else:
              T2[i,j]=-1
          else:
            T2[i,j]=-1
        else :
          T2[i,j]=-1
        # print (i,estado_decod, str_estado, T2[i,j])


In [148]:
v=0.2 # Factor de aprendizaje learning rate
y=0.8 # Factor de descuento discount factor

entrenar=0
s=0
Q1W=0
Q2W=0
Q1=np.zeros((len(Q),9),dtype=float)

turno=0

partida=1
while(entrenar<10):

 a=np.random.randint(9)  # Acción aleatoria al azar número entero en [0,8]
 if (turno % 2)==0: #turno del jugador 1
    print('Jugador 1 - Turno ',turno)
    print('-----------------------')
    Q1[s,a]=(1-v)*Q1[s,a]+v*(R[s]+y*max(Q1[siguiente,]))
    while T[s,a]==-1:
      a=np.random.randint(9)

      if estado_imposible (T,s)==True:
        print('estado imposible 1',s)
        s=0
    # T[s,a] es una transición posible
    siguiente=T[s,a] # Estado siguiente
    print('jugada 1-2',s,a,T[s,a])
    print('jugada 1-3',siguiente,Q[siguiente],decodificar_estado(Q[siguiente]))
    if estado_ganador(decodificar_estado(Q[siguiente]))==True:
        print('gana Q1')
        Q1W+=1
        s=0
        turno =0
        partida+=1
 else: #turno del jugador 2
    print('Jugador 2 - Turno ',turno)
    print('-----------------------')
    while T2[s,a]==-1:
      a=np.random.randint(9)
      if estado_imposible (T2,s)==True:
        print('estado imposible 2',s)
        s=1
    # T[s,a] es una transición posible
    siguiente=T2[s,a] # Estado siguiente
    print('jugada 2-2',s,a,T2[s,a])
    print('jugador 2-3',siguiente,Q[siguiente],decodificar_estado(Q[siguiente]))
    if estado_ganador(decodificar_estado(Q[siguiente]))==True:
      print('gana Q2')
      Q2W+=1
      s=0
      turno =0
      partida+=1

 s=siguiente
 print('turno',turno, Q[s])
 entrenar+=1
 turno+=1

Se han truncado las últimas 5000 líneas del flujo de salida.
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible 0 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
estado imposible 1 0
imposible

KeyboardInterrupt: 

In [118]:
print('partidas jugadas :',partida)
print('Ganadas jugador1 :',Q1W)
print('Ganadas jugador2 :',Q2W)
print('         Empates :',partida-Q1W-Q2W)



partidas jugadas : 1
Ganadas jugador1 : 0
Ganadas jugador2 : 0
         Empates : 1


In [139]:
# print(decodificar_estado(4094))
# print(Q[1399])
# print(dicc_inver[897])
# print (dicc[11221212])
print(T2[5301])
print(T[5463])
# print(np.base_repr(4094, base=3).zfill(9))
# print(221101212)


[  -1   -1   -1 5463   -1   -1   -1   -1   -1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1]
